# AnnData and WGCNA

This notebook creates a WGCNA object for each dataset and saves the results in the output folder.

The steps include:
1. Load the count matrix and metadata
2. Create a WGCNA object
3. Perform preprocessing
4. Find modules
5. Analyze WGCNA object
6. Save WGCNA results

## Packages

In [43]:
import sys
import psutil
import os
import PyWGCNA
import numpy as np
import pandas as pd
import anndata as ad
import wgcna.utils as rutils
import matplotlib.pyplot as plt
from memory_profiler import profile
from kneed import KneeLocator
from scipy.stats import median_abs_deviation

%load_ext autoreload
%autoreload 2

Running in Docker.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Settings

In [52]:
# Set matplotlib parameters
plt.rcParams['savefig.bbox'] = 'tight'

# Define count matrices path
count_matrices = "/vol/share/ranomics_app/count_matrices2"

# Define datasets
names = ['AC', 'PS', 'CS', 'EC', 'EG', 'HC', 'SP', 'TT']
names = ['AC']

# Output
output = "/vol/share/ranomics_app/output_new"

# Set save options
save_tom = False
save_adjacency_matrix = False
save_WGCNA = False
figure_type = 'png'

# Reduce the count matrix size?
reduce_matrix = False

## Processing

In [ ]:
print("Starting WGCNA for datasets:", names)
for name in names:
    output_path = f"{output}/{name}/"
    rutils.create_dir(output_path)

    # Prepare WGCNA object
    print("Preparing WGCNA object for", name, "dataset...")
    print("Input matrix:", input_matrix)
    print("Name:", name)
    print("Output path:", output_path)
    print("Save TOM:", save_tom)
    print("Save adjacency matrix:", save_adjacency_matrix)
    print("Save WGCNA results:", save_WGCNA)
    print("Reduce matrix:", reduce_matrix)

    species = rutils.find_species_by_initials(name.split("_")[0])

    # Create pyWGCNA object using the matrix and metadata files
    print("Creating pyWGCNA object...")
    pyWGCNA_obj = PyWGCNA.WGCNA(
        name=name,
        species=species,
        anndata=rutils.create_anndata(name),
        outputPath=output_path,
        figureType=figure_type,
        save=True
    )
    
    # Set colors for metadata
    unique_stages = pyWGCNA_obj.datExpr.obs["tissue"].unique()
    color_dict = rutils.generate_ranomics_color_dict(unique_stages)
    pyWGCNA_obj.setMetadataColor("tissue", color_dict)

    # Start WGCNA analysis
    print("Starting WGCNA analysis...")

    # Preprocessing
    print("Preprocessing data...")
    pyWGCNA_obj.preprocess()

    # Find modules
    print("Finding modules...")
    pyWGCNA_obj.findModules()

    print("Analyzing WGCNA object...")
    pyWGCNA_obj.analyseWGCNA()

    # Prepare and save WGCNA
    print("Preparing and saving WGCNA results...")
    tom = pyWGCNA_obj.TOM

    # Knee threshold & Edgelist export
    knee = rutils.detect_tom_threshold_from_array(
        tom, 
        plot_path=f"{output_path}figures/knee.{figure_type}"
    )
    if knee is not None:
        rutils.export_tom_edges_to_parquet(
            tom, 
            knee, 
            f"{output_path}{name}.parquet"
        )
    else:
        print("No knee detected, edge export skipped.")
    
    # Store additional metadata
    hub_genes = pyWGCNA_obj.top_n_hub_genes(n=100)
    hub_dict = {color: data for color,
                data in hub_genes.groupby('module_colors')}
    for color, df in hub_dict.items():
        df.index = df.index.get_level_values(1)
        
    adata.obsm["eigengenes"] = pyWGCNA_obj.datME
    adata.varm["eigengenes"] = pyWGCNA_obj.signedKME

    adata.uns["name"] = name
    adata.uns["output_path"] = os.path.abspath(pyWGCNA_obj.outputPath)
    adata.uns["power"] = pyWGCNA_obj.power
    adata.uns["species"] = pyWGCNA_obj.species
    adata.uns["network_type"] = pyWGCNA_obj.networkType
    adata.uns["TOM_type"] = pyWGCNA_obj.TOMType
    adata.uns["min_module_size"] = pyWGCNA_obj.minModuleSize
    adata.uns["sft"] = pyWGCNA_obj.sft.astype(float)
    adata.uns["gene_tree"] = pyWGCNA_obj.geneTree
    adata.uns["metadata_corr"] = {
        "module_corr": pyWGCNA_obj.moduleTraitCor, "module_p": pyWGCNA_obj.moduleTraitPvalue}
    adata.uns["hub_genes"] = hub_dict
    adata.uns["num_transcripts"] = pyWGCNA_obj.geneExpr.shape[1]

    # Drop dynamicColors column if it exists
    if 'dynamicColors' in adata.var:
        adata.var.drop(columns='dynamicColors', inplace=True)

    sc.pp.calculate_qc_metrics(adata, inplace=True)
    # Drop n_cells_by_counts column
    adata.var.drop(columns="n_cells_by_counts", axis=1, inplace=True)

    # Round float columns to n decimal places
    float_cols = adata.var.select_dtypes(include=['float64']).columns
    adata.var[float_cols] = adata.var[float_cols].round(n)

    # Save the annotated adata and ortho data
    adata.write_h5ad(f"{output_path}{name}.h5ad")

    print(f"Finished WGCNA for {name} dataset.")
    print("="*70)

print("Finished WGCNA for all datasets.")


Starting WGCNA for datasets: ['AC']
Directory already exists at: /vol/share/ranomics_app/output_new/AC/
Preparing WGCNA object for AC dataset...
Input matrix: /vol/share/ranomics_app/count_matrices2/AC.isoform.TMM.EXPR.matrix
Name: AC
Output path: /vol/share/ranomics_app/output_new/AC/
Save TOM: False
Save adjacency matrix: False
Save WGCNA results: False
Reduce matrix: False
Creating pyWGCNA object...
AnnData: AnnData object with n_obs × n_vars = 11 × 41063
    obs: 'tissue'
    var: 'go_terms', 'n_go_terms', 'ortho_id', 'ortho_count', 'ipr_id', 'ipr_desc', 'pfam_id', 'pfam_desc', 'tf_family', 'ecnumber', 'genename', 'description', 'description_long'
Filtered AnnData: AnnData object with n_obs × n_vars = 11 × 6606
    obs: 'tissue'
    var: 'go_terms', 'n_go_terms', 'ortho_id', 'ortho_count', 'ipr_id', 'ipr_desc', 'pfam_id', 'pfam_desc', 'tf_family', 'ecnumber', 'genename', 'description', 'description_long'
Final AnnData shape: (11, 6606)
Saving data to be True, checking requirements 

## Create AnnData including Metadata

### Settings

In [21]:
name = "AC"
go_path = f"/vol/share/ranomics_app/uniprot/{name}.uniprot.goslim_plant.gaf"
ortho_path = "/vol/share/ranomics_app/Ortho_Outgroup/N0.tsv"
ipr_path = f"/vol/share/ranomics_app/interpro/{name}.iprid.tsv"
pfam_path = f"/vol/share/ranomics_app/pfam/{name}.pfam.tsv"
tf_path = f"/vol/share/ranomics_app/TFs/{name}.TF.csv"
up_path = f"/vol/share/ranomics_app/uniprot/{name}.faa.annotation.csv"

### Create AnnData

In [23]:
if name == 'EC':
    input_matrix = f"/vol/share/ranomics_app/count_matrices2/{name}.isoform.TMM.EXPR.matrix.updated"
else:
    input_matrix = f"/vol/share/ranomics_app/count_matrices2/{name}.isoform.TMM.EXPR.matrix"
    
count_df = rutils.transform_count_matrix(input_matrix)
metadata_df = rutils.map_tissue_types(count_df)

count_df = count_df.set_index('sample')
metadata_df = metadata_df.set_index('sample')

metadata_df = metadata_df.loc[count_df.index]

adata = ad.AnnData(
    X=count_df.values,              # Werte-Matrix
    obs=metadata_df,                # Sample-Metadaten
    var=pd.DataFrame(index=count_df.columns)   # Transkriptnamen als var.index
)

display(adata)

AnnData object with n_obs × n_vars = 11 × 41063
    obs: 'tissue'

### Add Metadata

In [24]:
rutils.add_go_terms_to_anndata(adata, go_path)
rutils.add_ortho_id_to_anndata(adata, ortho_path, name)
rutils.add_ipr_columns(adata, ipr_path)
rutils.add_pfam_columns(adata, pfam_path)
rutils.add_tf_columns(adata, tf_path)
rutils.add_metadata_uniprot(adata, up_path)

display(adata)

IDs in adata.var that are missing in the Uniprot file: Index([], dtype='object')


AnnData object with n_obs × n_vars = 11 × 41063
    obs: 'tissue'
    var: 'go_terms', 'n_go_terms', 'ortho_id', 'ortho_count', 'ipr_id', 'ipr_desc', 'pfam_id', 'pfam_desc', 'tf_family', 'ecnumber', 'genename', 'description', 'description_long'

In [49]:
rutils.create_anndata("AC")

AnnData: AnnData object with n_obs × n_vars = 11 × 41063
    obs: 'tissue'
    var: 'go_terms', 'n_go_terms', 'ortho_id', 'ortho_count', 'ipr_id', 'ipr_desc', 'pfam_id', 'pfam_desc', 'tf_family', 'ecnumber', 'genename', 'description', 'description_long'
Loaded array in 0.01 s
Expression filter in 0.00 s
Variance filter in 0.02 s
Always-keep union in 4.84 s
Subset AnnData in 0.02 s
After filtering: 6606 genes retained (6606 total, 1785 always kept).
Total filtering time: 4.89 s
Filtered AnnData: AnnData object with n_obs × n_vars = 11 × 41063
    obs: 'tissue'
    var: 'go_terms', 'n_go_terms', 'ortho_id', 'ortho_count', 'ipr_id', 'ipr_desc', 'pfam_id', 'pfam_desc', 'tf_family', 'ecnumber', 'genename', 'description', 'description_long'
Final AnnData shape: (11, 6606)


AnnData object with n_obs × n_vars = 11 × 6606
    obs: 'tissue'
    var: 'go_terms', 'n_go_terms', 'ortho_id', 'ortho_count', 'ipr_id', 'ipr_desc', 'pfam_id', 'pfam_desc', 'tf_family', 'ecnumber', 'genename', 'description', 'description_long'